<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/MicTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マイク入力の例（講義室用）

## 1. 準備

### (1) ソフトのインストール

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

### (3) ブラウザからマイクを使う設定


In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

## 2. 音声の取り込み

In [ ]:
audiofile = "audio.wav"
second = 5
print(f"Speak to your microphone {second} sec...")
record(second, audiofile)
print("Done!")

In [ ]:
# 波形プロットにより録音を確認
# UserWarning: PySoundFile failed. Trying audioread instead. が出るが、無視して問題ない
import librosa
import librosa.display
speech, rate = librosa.load(audiofile, sr=16000)
librosa.display.waveshow(speech, sr=rate)

In [ ]:
import IPython.display
IPython.display.Audio("audio.wav", autoplay=True)